## Convolutional Layer

In [1]:
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images()["images"]
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale=1/255)(images)

In [2]:
images.shape

TensorShape([2, 70, 120, 3])

In [3]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7)
fmaps = conv_layer(images)

In [4]:
fmaps.shape

TensorShape([2, 64, 114, 32])

In [5]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7, padding="same")
fmaps = conv_layer(images)
fmaps.shape

TensorShape([2, 70, 120, 32])

In [6]:
kernels, biases = conv_layer.get_weights()
kernels.shape

(7, 7, 3, 32)

In [7]:
biases.shape

(32,)

## Pooling Layer

In [8]:
max_pool = tf.keras.layers.MaxPool2D(pool_size=2)

In [9]:
# depthwise pooling
class DepthPool(tf.keras.layers.Layer):
    def __init__(self, pool_size=2, **kwargs):
        super().__init__(**kwargs)
        self.pool_size = pool_size

    def call(self, inputs):
        shape = tf.shape(inputs)
        groups = shape[-1] // self.pool_size
        new_shape = tf.concat([shape[:-1], [groups, self.pool_size]], axis=0)
        return tf.reduce_max(tf.reshape(inputs, new_shape), axis=-1)

In [10]:
# global average pooling layer
global_avg_pool = tf.keras.layers.GlobalAvgPool2D()
global_avg_pool = tf.keras.layers.Lambda(
    lambda X: tf.reduce_mean(X, axis=[1, 2])
)

In [11]:
global_avg_pool(images)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.643388  , 0.59718215, 0.5825038 ],
       [0.7630747 , 0.26010972, 0.10848834]], dtype=float32)>

## Structure of CNN

In [12]:
from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding='same',
                        activation='relu', kernel_initializer='he_normal')
model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu',
                          kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation='relu',
                          kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation='softmax'),
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## ResNet-34

In [13]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding='same', kernel_initializer='he_normal', use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [14]:
model = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
])
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

## Pre-trained Model

In [15]:
model = tf.keras.applications.resnet50.ResNet50(weights='imagenet')

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [16]:
import numpy as np
images = load_sample_images()["images"]
images = np.array(images)
images_resized = tf.keras.layers.Resizing(height=224, width=224,
                                          crop_to_aspect_ratio=True)(images)

In [17]:
inputs = tf.keras.applications.resnet50.preprocess_input(images_resized)
Y_proba = model.predict(inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step


In [18]:
Y_proba.shape

(2, 1000)

In [19]:
top_K = tf.keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print(f"Image #{image_index}")
    for class_id, name, y_proba in top_K[image_index]:
        print(f"  {class_id} - {name:12s} {y_proba:.2%}")

35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
Image #0
  n03598930 - jigsaw_puzzle 38.05%
  n02782093 - balloon      14.30%
  n03877845 - palace       4.83%
Image #1
  n04209133 - shower_cap   18.77%
  n02782093 - balloon      12.99%
  n09229709 - bubble       11.91%


## Transfer Learning using Pre-trained Model

In [20]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [21]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True
)

In [22]:
batch_size = 32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])
train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

In [23]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=42),
    tf.keras.layers.RandomRotation(factor=0.05, seed=42),
    tf.keras.layers.RandomContrast(factor=0.2, seed=42),
])

In [24]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet',
                                                     include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation='softmax')(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [25]:
for layer in base_model.layers:
    layer.trainable = False

In [28]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])
history = model.fit(train_set, epochs=3, validation_data=valid_set)

Epoch 1/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 254ms/step - accuracy: 0.7138 - loss: 1.0285 - val_accuracy: 0.8276 - val_loss: 0.6022
Epoch 2/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 147ms/step - accuracy: 0.8982 - loss: 0.4328 - val_accuracy: 0.8530 - val_loss: 0.7818
Epoch 3/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 22s 155ms/step - accuracy: 0.9448 - loss: 0.1752 - val_accuracy: 0.8784 - val_loss: 0.5745


In [29]:
for layer in base_model.layers[56:]:
    layer.trainable = True

In [30]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])
history = model.fit(train_set, epochs=10, validation_data=valid_set)

Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 59s 402ms/step - accuracy: 0.8731 - loss: 0.3900 - val_accuracy: 0.8457 - val_loss: 0.6524
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 26s 292ms/step - accuracy: 0.9864 - loss: 0.0610 - val_accuracy: 0.9002 - val_loss: 0.3928
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 275ms/step - accuracy: 0.9917 - loss: 0.0235 - val_accuracy: 0.9002 - val_loss: 0.4805
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 25s 278ms/step - accuracy: 0.9931 - loss: 0.0160 - val_accuracy: 0.9129 - val_loss: 0.4040
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 274ms/step - accuracy: 0.9934 - loss: 0.0136 - val_accuracy: 0.9038 - val_loss: 0.3848
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 42s 285ms/step - accuracy: 0.9982 - loss: 0.0078 - val_accuracy: 0.9165 - val_loss: 0.3196
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 25s 281ms/step - accuracy: 0.9939 - loss: 0.0117 - val_accuracy: 0.9074 - val_loss: 0.4055
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 40s 274ms/step - accuracy: 0.9993 - loss: 0.0032 - val_accu

## Classification and Location Prediction

In [32]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet',
                                                     include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = tf.keras.layers.Dense(n_classes, activation='softmax')(avg)
loc_output = tf.keras.layers.Dense(4)(avg)
model = tf.keras.Model(inputs=base_model.input,
                       outputs=[class_output, loc_output])
model.compile(loss=['sparse_categorical_crossentropy', 'mse'],
              loss_weights=[0.8, 0.2], optimizer=optimizer, metrics=['accuracy'])